<div style="background-color:#e6ffe6; padding:10px; color:black; border-style:
solid;; border-color:#00e600; border-width:1px; color:black;">

# AST4310 2025, Project 3

</div>

Authored by: *(Replace this text with your candidate numbers)*.

In [1]:
# Your imports here

### Exercise 1: Non-LTE Radiative Transfer with the Feautrier method and $\mathbf{\Lambda}$ iteration [45 points]

The Feautrier method consists of solving the equation

\begin{equation}
\mathbf{T}P = S,
\end{equation}

where $\mathbf{T}$ is a tridiagonal matrix. The function `Tmatrix` computes $\mathbf{T}$ for a single frequency and direction on a 1D plane-parallel atmosphere, given an optical depth scale and $\mu$. 

The source function for a two-level atom with coherent scattering can be written as

$$
S = (1-\varepsilon)\mathbf{\Lambda}[S] + \varepsilon B,
$$
and a direct solution can be written as
$$
S = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda})^{-1}[\varepsilon B],
$$

which involves inverting matrices (impractical for most real-life applications). Other numerical methods to solve for $S$ are the classical $\Lambda$ iteration:

$$
S^{(n+1)} = (1-\varepsilon)\mathbf{\Lambda}[S^{(n)}] + \varepsilon B,
$$

where we need a guess for $S^{(1)}$, often $S^{(1)}=B.$ An alternative that is much more efficient is the approximate (or accelerated) $\Lambda$ iteration:

$$
S^{(n+1)} = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda}^*)^{-1}\left[S^\mathrm{FS}-(1-\varepsilon)\mathbf{\Lambda}^*[S^{(n)}]\right].
$$

For the iterative schemes, we can measure the convergence speed by calculating the fractional difference between source functions of successive iterations, e.g.:
$$
\delta = \left|\left|\frac{S_\nu^{(n+1)}-S_\nu^{(n)}}{S_\nu^{(n)}}\right| \right|.
$$

Below we use the convention that 1 is the first element of an array, and $n$ the last.

<div style="background-color:#e6ffe6; padding:10px; color:black;  border-style:
solid;; border-color:#00e600; border-width:1px; color:black;">
    
* *[5 points]* At the top of the atmosphere, we can assume $I^-=0$. Why? Using a Taylor expansion around the Feautrier $P_1$ and assuming $I^-=0$ at the top of the atmosphere, show that the boundary coefficients of $\mathbf{T}$ at the top can be written as:

$$
\begin{aligned}
B_1 &= \frac{2}{\Delta\tau^2} + \frac{2}{\Delta\tau} + 1, \\ 
C_1 &= \frac{2}{\Delta\tau^2}, \\
\Delta\tau &=\tau_{2} - \tau_{1}.
\end{aligned}
$$ 

* *[6 points]* At the bottom of the atmosphere, we can assume $I^+=S$. Why? Using a Taylor expansion around the Feautrier $P_{n}$, and assuming $I^+=S$ at the bottom of the atmosphere, show that the boundary coefficients of $\mathbf{T}$ at the bottom can be written as:

$$
\begin{aligned}
A_{n} &= \frac{2}{\Delta\tau(\Delta\tau+2)},  \\ 
B_{n} &= \frac{2+2\Delta\tau +\Delta\tau^2}{\Delta\tau(\Delta\tau+2)},\\
\Delta\tau &=\tau_{n} - \tau_{n-1}.
\end{aligned}
$$

* *[6 points]* Using the function `Λ_matrix()` and `tau = logrange(1e-4, 1e2, 50)`, plot the $\mathbf{\Lambda}$ matrix and discuss its physical meaning. Explore the effect of using a different number of points in the Gaussian quadrature. What is a reasonable number of points?

* *[6 points]* The function `solve_cs_direct()`, which takes as arguments $\tau$, $B$, and $\varepsilon$, computes $S$ and $J$ using a direct solution for the problem of coherent scattering in a two-level atom (using matrix inversions). Use it to solve the problem for $\varepsilon=10^{-3}$, `tau = logrange(1e-4, 1e4, 50)` and $B=1.5\tau$. Do a log-log plot for S, J, and B. In this case, why is $J>B$ at the surface?
    
* *[9 points]* Write a function `solve_cs_CLI()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using the classical $\Lambda$ iteration (CLI). For the simple case where $B=1$ everywhere, $\varepsilon=10^{-2}$, plot $S$, $J$, $B$, for CLI and the direct solution from `solve_cs_direct()`. Does CLI work well for this case? How many iterations do you need to achieve $\delta < 10^{-3}$?
    
* *[13 points]* Write a function `solve_cs_ALI()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using the accelerated $\Lambda$ iteration (ALI). For the $\mathbf{\Lambda}^*$ operator, use the OAB operator, which simply uses the diagonal of the full $\mathbf{\Lambda}$ operator. Usually, this $\mathbf{\Lambda}^*$ is computed without having to build the $\mathbf{\Lambda}$ matrix explicitly, during the tridiagonal solution of $\mathbf{T}P = S$. Use the provided function `diag_inverse_tri()` to obtain $\mathbf{\Lambda}^*$ from the inverse of $\mathbf{T}$ matrices, computed using `Tmatrix()`. For the simple case where $\varepsilon=10^{-4}$, `tau = logrange(1e-4, 1e4, 50)`, `B = reverse(tau)*10 .+ tau ./ 1000`, plot $S$ and $B$, for $S$ computed with the direct solution, CLI, and ALI, both with a maximum of 100 iterations. Discuss the differences. How many iterations of ALI do you need to be close to the direct solution? And how many iterations of CLI?
    
</div>

In [2]:
# Your answers here

### Exercise 2: Zeeman effect and Polarisation [30 points]

<div style="background-color:#e6ffe6; padding:10px; color:black; border-style:
solid;; border-color:#00e600; border-width:1px; color:black;">
    
* *[5 points]* Assume that the terms $^5P$ and $^5D$  can define respectively the upper and lower level of a bound-bound transition. How many different levels does each term have, and how many permited transitions are possible with Zeeman splitting? Remember that a level is set by a fixed S, L, and J. The quantum mechanical selection rules for electric dipole transitions are $\Delta M_j=0, \pm 1$ and $\Delta J = 0, \pm 1$ (as long as $J^u$ and $J^l$ are not both zero). 

* *[10 points]* Modify the function `unno_rachkovsky()` so that it works for arbitrary Zeeman patterns of the anomalous Zeeman effect.

* *[7 points]* The table below has some that data you will need to compute the Stokes vector using the Unno-Rachkovsky solution for two different lines, Fe I 617 nm and Ti I 2221 nm. Assume that both lines form in a region with typical temperatures of $T\approx 5060$ K. Let's start with the Fe I line, which exhibits normal Zeeman effect and is widely used to measure stellar magnetic fields. Using the assumptions from the table below, calculate the Stokes vector for the Fe I line assuming a vertical magnetic field. What value of B would you need to see the split between $\sigma_r$ and $\sigma_b$ in the intensity profile? 

* *[8 points]* As above, use Unno-Rachkovsky to obtain the Stokes vector for the Ti I line, which exhibits anomalous Zeeman effect. Include all Zeeman components. What value of B would you need to see the split between $\sigma_r$ and $\sigma_b$ in the intensity profile? What value of B would you need to see the split among the individual $\sigma_b$ components (not just between $\sigma_r$ and $\sigma_b$)?

|                  | Fe I 617 | Ti I 2221 |
|------------------|-----------:|-----------:|
| Air wavelength (nm) |   617.33 | 2221.728  |  
| Lower level       | 4s $^5P_1$ | 4s $^5P_1$ |
| Upper level       | 4p  ${}^5D_0$ | 4p  ${}^5D_1$|
| $S_0$ (kW m $^2$ nm $^{-1}$ sr $^{-1}$) | 10 | 0.74 |
| $S_1$ (kW m $^2$ nm $^{-1}$ sr $^{-1}$)  | 22  | 0.57 |
| $\eta$            | 300    | 100 |
| $a$               | 0.03    | 0.07  |
| Atomic mass (u)   | 55.845 | 47.867 |


</div>

In [3]:
# Your answers here

### Exercise 3: Measuring magnetic fields [20 points]

<div style="background-color:#e6ffe6; padding:10px; color:black; border-style:
solid;; border-color:#00e600; border-width:1px; color:black;">

Now you will use the theory to perform a practical measurement: magnetic fields in sunspots. Use the SST observations from the file `SST_sunspot_617_2020.08.07.jld2`, which contains Stokes I and V.

* *[15 points]* Using the Weak Field Approximation, measure the line-of-sight component of the magnetic field ($B\cos\gamma$) for every spatial location in the observations. Plot the resulting map, using units of mT. What can you say about the magnetic field in sunspots? Compare with a map of line-of-sight velocity and discuss.

* *[5 points]* Assuming that you had observed the full Stokes vector (I, Q, U, V), how would you derive the full magnetic field vector from the observations?
</div>

In [4]:
# Your answers here